In [4]:
%pip install pgeocode tqdm pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import pgeocode
from tqdm import tqdm

# 1. Загружаем исходник
df_source = pd.read_csv('../data/data.csv')

def clean_zip(x):
    x = str(x)
    if x == '--' or pd.isna(x): return '0'
    return x.split('-')[0]

unique_zips = df_source['zipcode'].apply(clean_zip).unique()
print(f"Найдено {len(unique_zips)} уникальных индексов.")

# 2. Инициализируем движок (US)
nomi = pgeocode.Nominatim('us')

# 3. Сбор данных (pgeocode умеет принимать список сразу!)
# Это будет работать ГОРАЗДО быстрее, чем цикл
print("Запрос данных...")
# Конвертируем все зипкоды в строки
zip_list = [str(z) for z in unique_zips if len(str(z)) >= 3]

# query_postal_code возвращает DataFrame сразу
geo_data = nomi.query_postal_code(zip_list)

# 4. Приводим к вашему формату
# pgeocode возвращает колонки: postal_code, country_code, place_name, state_name, county_name, latitude, longitude
df_new_geo = geo_data[['postal_code', 'place_name', 'county_name', 'state_code', 'latitude', 'longitude']].copy()

df_new_geo.columns = ['Индекс', 'Город', 'Округ', 'Штат_код', 'Широта', 'Долгота']

# Удаляем пустые (где индекс не найден)
df_new_geo = df_new_geo.dropna(subset=['Широта'])

# Индекс должен быть int для совместимости
df_new_geo['Индекс'] = df_new_geo['Индекс'].astype(int)

print(f"Сгенерировано {len(df_new_geo)} записей.")
print(df_new_geo.head())

# 5. Сохраняем
df_new_geo.to_csv('../assets/df_pars_v2.csv', index=False)
print("Файл сохранен.")

Найдено 4319 уникальных индексов.
Запрос данных...
Сгенерировано 4176 записей.
   Индекс           Город        Округ Штат_код   Широта   Долгота
0   28387  Southern Pines        Moore       NC  35.1697  -79.3957
1   99216         Spokane      Spokane       WA  47.6634 -117.2193
2   90049     Los Angeles  Los Angeles       CA  34.0660 -118.4740
3   75205          Dallas       Dallas       TX  32.8260  -96.7843
4   32908        Palm Bay      Brevard       FL  27.9816  -80.6894
Файл сохранен.


In [3]:
# 1. Загружаем старый файл
df_old = pd.read_csv('../assets/df_pars.csv') # Путь к текущему файлу

# 2. Мерджим старый и новый по Индексу
comparison = df_new_geo.merge(df_old, on='Индекс', suffixes=('_new', '_old'), how='inner')

# 3. Считаем разницу в координатах
comparison['diff_lat'] = abs(comparison['Широта_new'] - comparison['Широта_old'])
comparison['diff_lon'] = abs(comparison['Долгота_new'] - comparison['Долгота_old'])

# 4. Смотрим расхождения
# Порог 0.1 градуса ~ 11 км. Если больше - это серьезная ошибка.
big_errors = comparison[(comparison['diff_lat'] > 0.1) | (comparison['diff_lon'] > 0.1)]

print(f"Всего общих индексов: {len(comparison)}")
print(f"Индексов с серьезными ошибками координат (>10км): {len(big_errors)}")

if len(big_errors) > 0:
    print("\nПримеры ошибок:")
    display(big_errors[['Индекс', 'Город_new', 'Город_old', 'Широта_new', 'Широта_old']].head())
    print(big_errors[['Индекс', 'Город_new', 'Город_old', 'Широта_new', 'Широта_old']].head())
else:
    print("Старый файл удивительно точен!")

Всего общих индексов: 4097
Индексов с серьезными ошибками координат (>10км): 84

Примеры ошибок:


,Индекс,Город_new,Город_old,Широта_new,Широта_old
130,79928,El Paso,El Paso,31.6631,31.7586
134,79938,El Paso,El Paso,31.8045,31.8238
146,28278,Charlotte,Charlotte,35.2072,35.0985
197,85387,Surprise,Sun City,33.7129,33.6306
202,32413,Panama City Beach,Panama City Beach,30.3105,30.1847


     Индекс          Город_new          Город_old  Широта_new  Широта_old
130   79928            El Paso            El Paso     31.6631     31.7586
134   79938            El Paso            El Paso     31.8045     31.8238
146   28278          Charlotte          Charlotte     35.2072     35.0985
197   85387           Surprise           Sun City     33.7129     33.6306
202   32413  Panama City Beach  Panama City Beach     30.3105     30.1847
